In [1]:
#Importing modules
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
#Importing data

merged_train = pd.read_csv("merged_data.csv")
merged_train.columns
df = merged_train.copy()
del df['id']
df.columns

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'dcoilwtico',
       'city', 'state', 'type', 'cluster', 'holiday'],
      dtype='object')

In [3]:
# Drop the 'date' column as it may not be useful for prediction
df = df.drop('date', axis=1)

# Convert categorical variables to numerical using one-hot encoding
df = pd.get_dummies(df)

# Split the data into features (X) and target variable (y)
X = df.drop('sales', axis=1)
y = df['sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the KNN regressor
knn = KNeighborsRegressor(n_neighbors=5)

# Train the model
knn.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = knn.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

# Now you can use the trained model to make predictions on new data
# For example, you can use knn.predict(new_data_scaled) where new_data_scaled is a new set of features

Root Mean Squared Error: 414.2786222308964


In [4]:
y_pred


array([2270.2   ,  232.    ,  363.7246, ...,  251.4506,  614.518 ,
        894.6844])

In [5]:
test = pd.read_csv("test.csv")
oil = pd.read_csv("oil.csv")

# Forward fill missing values
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='ffill')

# Backward fill missing values for the first pass
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')

# Backward fill again for the last pass, in case the first value is missing
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')

# If somehow there's still missing values, fill them with the mean
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(value=oil['dcoilwtico'].mean())

merged_test = pd.merge(test, oil, how='left', on='date')
merged_test['dcoilwtico'].fillna(value=oil['dcoilwtico'].mean(), inplace=True)
merged_test
merged_train[merged_train.isna().any(axis=1)]

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster,holiday


In [6]:
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv")
holidays_events = pd.read_csv("holidays_events.csv")
merged_test = pd.merge(merged_test, stores, how='left', on='store_nbr')
merged_test

,id,date,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,46.80,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,46.80,Quito,Pichincha,D,13
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,Quito,Pichincha,D,13
4,3000892,2017-08-16,1,BOOKS,0,46.80,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,Quito,Pichincha,B,6
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,Quito,Pichincha,B,6
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,Quito,Pichincha,B,6
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,Quito,Pichincha,B,6


In [7]:
merged_test[merged_test.isna().any(axis=1)]
merged_test.value_counts('city')
merged_test['holiday'] = False
for ind in holidays_events.index:
    holiday_local = holidays_events['locale_name'][ind]
    row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
    row_i = row.index
    merged_test.loc[row_i, 'holiday'] = True

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

...

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == holiday_local) | (merged_train['state'] == holiday_local))]
C:\Users\joona\AppData\Local\Temp\ipykernel_7860\3927819753.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  row = merged_test[(merged_test['date'] == holidays_events['date'][ind]) & (holidays_events['transferred'][ind] == False) & ((merged_train['city'] == h

In [8]:
merged_test.head

<bound method NDFrame.head of             id        date  store_nbr                      family  \
0      3000888  2017-08-16          1                  AUTOMOTIVE   
1      3000889  2017-08-16          1                   BABY CARE   
2      3000890  2017-08-16          1                      BEAUTY   
3      3000891  2017-08-16          1                   BEVERAGES   
4      3000892  2017-08-16          1                       BOOKS   
...        ...         ...        ...                         ...   
28507  3029395  2017-08-31          9                     POULTRY   
28508  3029396  2017-08-31          9              PREPARED FOODS   
28509  3029397  2017-08-31          9                     PRODUCE   
28510  3029398  2017-08-31          9  SCHOOL AND OFFICE SUPPLIES   
28511  3029399  2017-08-31          9                     SEAFOOD   

       onpromotion  dcoilwtico   city      state type  cluster  holiday  
0                0       46.80  Quito  Pichincha    D       13    F

In [9]:
merged_test.value_counts('holiday')
merged_test.head
idcolumn = merged_test['id'].copy()
merged_test_filled=merged_test.fillna(merged_test.mean())
del merged_test_filled['id']
print(idcolumn.head(50))
merged_test_filled.head
y_test=knn.predict(X_test_scaled)
y_test

C:\Users\joona\AppData\Local\Temp\ipykernel_7860\346020588.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_test_filled=merged_test.fillna(merged_test.mean())


0     3000888
1     3000889
2     3000890
3     3000891
4     3000892
5     3000893
6     3000894
7     3000895
8     3000896
9     3000897
10    3000898
11    3000899
12    3000900
13    3000901
14    3000902
15    3000903
16    3000904
17    3000905
18    3000906
19    3000907
20    3000908
21    3000909
22    3000910
23    3000911
24    3000912
25    3000913
26    3000914
27    3000915
28    3000916
29    3000917
30    3000918
31    3000919
32    3000920
33    3000921
34    3000922
35    3000923
36    3000924
37    3000925
38    3000926
39    3000927
40    3000928
41    3000929
42    3000930
43    3000931
44    3000932
45    3000933
46    3000934
47    3000935
48    3000936
49    3000937
Name: id, dtype: int64


array([2270.2   ,  232.    ,  363.7246, ...,  251.4506,  614.518 ,
        894.6844])

In [10]:
print(y_test)
print("id,sales")
print(len(idcolumn))
lines=["id,sales \n"]
file=open("submission_test.txt", "w")
file.writelines(lines)
for i in range(len(idcolumn)):
    file.writeline(str(idcolumn[i])+","+str(y_test[i])+"\n")
file.close

[2270.2     232.      363.7246 ...  251.4506  614.518   894.6844]
id,sales
28512


AttributeError: '_io.TextIOWrapper' object has no attribute 'writeline'